In [56]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import ctypes
from torch.autograd import Variable
import json
import cv2
from torch.utils import data
from pycocotools.coco import COCO
import math
from torch.utils.data import DataLoader
from unet_utils.dice_score import dice_loss


from FAUNet_Train import channel_normalize
from FAUNet_Train import myDataset
from FAUNet_Train import DoubleConv,Down,Up,OutConv
from FAUNet_Train import FAUNet
from FAUNet_Train import mynormalize

import os

import numpy as np
import matplotlib.pyplot as plt

import cv2
import numpy as np

def resize_image_cv2(image_array, target_size=(256, 256)):
    """
    Resize an image to a target size using OpenCV.
    
    Parameters:
    - image_array: numpy array of shape (height, width, channels).
    - target_size: tuple, target size in (width, height).
    
    Returns:
    - Resized image as a numpy array.
    """
    resized_image = cv2.resize(image_array, target_size, interpolation=cv2.INTER_AREA)
    return resized_image

Device = 'cuda:0'   


activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook


main_path = '/home/geospatial/FAUNet/data_set/'

epoch = 96

### set paths
weights_folder = "/home/geospatial/FAUNet/FAUNET_weights/"
PATH = weights_folder + "/weights_" + str(epoch) + "epoch_final.pt"
#print('loading model: ', PATH)
# resfolder = './unet_res_lowres_full/'

#####
# faunet = FAUNet(3,2,Device)
faunet = torch.load(PATH)
# faunet.load_state_dict(PATH)
faunet.eval()
faunet.to(torch.device(Device))


# this is the folder where your test images are 
pth_images_test = "/home/geospatial/FAUNet/numpyarrays"

# this is the folder where the results will be written to
resfolder = '/home/geospatial/FAUNet/data_set/test_results_india/'
os.makedirs(resfolder,exist_ok= True)


file_list = os.listdir(pth_images_test)[:1]

In [ ]:
for ii,file_ in enumerate(file_list):
    print(ii,file_)
    with torch.no_grad():
        im_name = str(ii) 
        img_dir = file_
        im = np.load(pth_images_test + '/' + img_dir).transpose(1,2,0)[:,:,:]
        im = resize_image_cv2(im)
        
        im_h,im_w,im_c = im.shape
        if im_h == 256:
            if im_h == 256:
                im_transposed = np.transpose(im,(2,0,1))
                im_tensor = torch.tensor(im_transposed).float()     
                x =  im_tensor.to(torch.device(Device)).unsqueeze(0)
                c,e = faunet(x)
                seg = c.squeeze(0).cpu().numpy()
                edges = e.squeeze(0).cpu().numpy()       
                
                final_res =   1.0*(seg[1,:,:]>0)* (1 - 1.0*(edges[1,:,:]>0))
                
                # edges = 1.0*(edges[1,:,:]>0)
                cv2.imwrite(resfolder + im_name+ '.png',255*im)

                # The resulting extent mask
                cv2.imwrite(resfolder + im_name + '_resExtent.png',255.0*(seg[1,:,:]>0))
                # The resulting edge mask
                cv2.imwrite(resfolder + im_name+ '_resEdge.png',255.0*(edges[1,:,:]>0))
                final_res = final_res.astype('uint8')
                # this is the postprocessed result (close parcels)
                cv2.imwrite(resfolder + im_name+ '_final_res.png',255.0*final_res)
                
                fig,axs=plt.subplots(1,4,figsize=(15,15))
                axs[0].imshow(im)
                axs[1].imshow(seg[1,:,:]>0)
                axs[2].imshow(edges[1,:,:]>0)
                axs[3].imshow(final_res*255)
                for ax in axs.ravel():
                    ax.set_axis_off()
                plt.show()
                print()

In [57]:
from inference import get_prediction_map

In [58]:
for ii,file_ in enumerate(file_list):
    print(ii,file_)
    with torch.no_grad():
        im_name = str(ii) 
        img_dir = file_
        im = np.load(pth_images_test + '/' + img_dir).transpose(1,2,0)[:,:,:]
        im = resize_image_cv2(im)
        
        prediction=get_prediction_map(im,"/home/geospatial/FAUNet/FAUNET_weights/weights_96epoch_final.pt")
        print(prediction)
        

0 cluster_image_232_0.1_5_5_0.005.tif_tile_0.npy
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
